In [9]:
import collections
import csv
from itertools import islice

import numpy as np
import numpy.typing as npt

In [2]:
from pathlib import Path

In [3]:
Path().cwd()

PosixPath('/home/adoni5/Projects/Icarust_supplementary/barcoding/R10_signal')

In [4]:
PREFIX = (
    "TTTTTTTTTTTTTTTTTTAATCAAGCAGCGGAGTTGAGGACGCGAGACGGGACTTTTTTAGCAGACTTTACGGACTACGACT"
)

In [5]:
def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) --> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

In [7]:
with open("../working/model_test.tsv", "r") as fh:
    reader = csv.reader(fh, delimiter="\t")
    kmer_values = {x[0]: float(x[1]) for x in reader}

In [10]:
def signalify(
    kmers: dict[str, dict[str, float]], sequence: str
) -> npt.NDArray[np.int16]:
    """convert a given sequence to signal using R10 models,
    returning np array containing 10 samples per base"""
    a = []
    # Always upper case signal
    for kmer in sliding_window(sequence.upper(), 9):
        value = kmers["".join(kmer)]
        value = (value * 2048) / 200 - 0
        for _ in range(10):
            a.append(value)
    return np.array(a).astype(np.int16)

In [13]:
prefix_signal = signalify(kmer_values, PREFIX)

In [14]:
np.save("prefix.squiggle.npy", arr=prefix_signal)